In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('data_zpyYWs0.csv')
sample =pd.read_csv('sample_submission_Gk1Mne1.csv')

In [4]:
train.head()

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id
0,1,Pat Cummins,DNB,3,28,0,0,19 Oct 2011,v South Africa Centurion,1
1,2,Steve Smith,DNB,-,-,0,0,19 Oct 2011,v South Africa Centurion,1
2,10,Mitch Marsh,8*,1,19,1,0,19 Oct 2011,v South Africa Centurion,1
3,13,David Warner,20,-,-,0,0,19 Oct 2011,v South Africa Centurion,1
4,1,Pat Cummins,11*,1,73,0,0,23 Oct 2011,v South Africa Gqeberha,2


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2575 entries, 0 to 2574
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   player_id      2575 non-null   int64 
 1   player_name    2575 non-null   object
 2   runs_scored    2575 non-null   object
 3   wickets        2575 non-null   object
 4   runs_conceded  2575 non-null   object
 5   catches        2575 non-null   object
 6   stumpings      2575 non-null   object
 7   match_date     2575 non-null   object
 8   opposition     2575 non-null   object
 9   match_id       2575 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 201.3+ KB


In [92]:
#train[(train.runs_scored=='DNB') & (train.wickets=='-')]
#train[(train.runs_scored=='TDNB')].shape#& (train.wickets=='-')]

#train[(train.runs_scored=='TDNB') & (train.wickets=='-')]

In [7]:
train.loc[train.catches=='-'].shape

(0, 10)

Columns to be cleaned up:
1. runs_scored with DNB and TDNB(team did not bat)
    - to be replaced with -99 initially 
    - not out asterik can be removed and another column can be made.
        * Column with category match_play_status  [bowled, batted, bat_bowled not bowled and not batted]
        * For the TDNB where the match was abonded or the no result due to weather. We can make a column with complete_game with categories [yes no and reduced_overs]
        * For the not out player out_notout [yes or no] with * scores
2. wickets
    - '-' can be replace with 0 and can be captured in the match_play_status column.
3. runs_conceded
    - '-' can be replace with 0 and can be captured in the match_play_status column.
4. catches
    * we can see from the '-' when a match is abonded or no result is there we have a '-' for wickets, runs_conceded, catches and stumping.
    * for the column complete_game we have a new data that the game was not completed for the catches column
    * datapoints are there where all four columns are having '-'.
    * replace complete_game with no result and replace the value with 0.
5. match_date
    - covert the column into date format and extract the match age
6. opposition extract from the column.

         

In [10]:
train['runs_scored_1']= train['runs_scored']

f = lambda x: -99 if x=='DNB' or x =='TDNB' else x
train['runs_scored_1'] = train['runs_scored'].map(f)   
train['runs_scored_1']=train['runs_scored_1'].replace('\*','',regex=True).astype(int)                     

In [30]:
type(train['runs_scored'].str[-1])

pandas.core.series.Series

In [21]:
train.head()

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id,runs_scored_1,out_notout
0,1,Pat Cummins,DNB,3,28,0,0,19 Oct 2011,v South Africa Centurion,1,-99,0
1,2,Steve Smith,DNB,-,-,0,0,19 Oct 2011,v South Africa Centurion,1,-99,0
2,10,Mitch Marsh,8*,1,19,1,0,19 Oct 2011,v South Africa Centurion,1,8,1
3,13,David Warner,20,-,-,0,0,19 Oct 2011,v South Africa Centurion,1,20,0
4,1,Pat Cummins,11*,1,73,0,0,23 Oct 2011,v South Africa Gqeberha,2,11,1


In [36]:
# 0 -> out ; 1 -> not out ; 2 -> DNB or TDNB
train['score_temp'] = train['runs_scored'].str[-1]

def f(x):
    if x['runs_scored'] =='TDNB' or x['runs_scored'] =='DNB': return 2
    elif x['score_temp'] =='*': return 1
    else: return 0
    
    #x.drop('score_temp', axis=1, drop=TRUE)

#train['out_notout'] = train['runs_scored'].str.contains('\*').astype(int)
train['out_notout'] = train.apply(f, axis=1)
train.drop('score_temp', axis=1, inplace=True)
train.head()

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id,runs_scored_1,out_notout
0,1,Pat Cummins,DNB,3,28,0,0,19 Oct 2011,v South Africa Centurion,1,-99,2
1,2,Steve Smith,DNB,-,-,0,0,19 Oct 2011,v South Africa Centurion,1,-99,2
2,10,Mitch Marsh,8*,1,19,1,0,19 Oct 2011,v South Africa Centurion,1,8,1
3,13,David Warner,20,-,-,0,0,19 Oct 2011,v South Africa Centurion,1,20,0
4,1,Pat Cummins,11*,1,73,0,0,23 Oct 2011,v South Africa Gqeberha,2,11,1


In [37]:
def f(x):
    if x['runs_scored'] =='TDNB': return 0
    elif x['runs_scored'] =='DNB' and x['catches'] =='-' and x['wickets'] =='-' and x['runs_conceded'] =='-': return 0
    else: return 1

train['is_match_complete']= train.apply(f, axis=1)


In [ ]:
train[(train.wickets=='-') & (train.runs_conceded!='-')].head()
# no matches with - wichets and runs conceded not qual to -

In [38]:
# Create column batted and bowled
def f(x):
    if x['wickets'] =='-' and x['runs_conceded'] == '-': return 0
    else: return 1

train['bowled_in_match']= train.apply(f, axis=1)

In [106]:
train[(train.runs_scored=='-') ].head()
# no matches with - runs scored

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id,runs_scored_1,out_notout,is_match_complete,bowled_in_match


In [39]:
# Create column batted and bowled
def f(x):
    if x['runs_scored'] =='TDNB' or x['runs_scored'] =='DNB': return 0
    else: return 1

train['batted_in_match']= train.apply(f, axis=1)

In [108]:
train.head()

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id,runs_scored_1,out_notout,is_match_complete,bowled_in_match,batted_in_match
0,1,Pat Cummins,DNB,3,28,0,0,19 Oct 2011,v South Africa Centurion,1,0,0,1,1,0
1,2,Steve Smith,DNB,-,-,0,0,19 Oct 2011,v South Africa Centurion,1,0,0,1,0,0
2,10,Mitch Marsh,8*,1,19,1,0,19 Oct 2011,v South Africa Centurion,1,8,1,1,1,1
3,13,David Warner,20,-,-,0,0,19 Oct 2011,v South Africa Centurion,1,20,0,1,0,1
4,1,Pat Cummins,11*,1,73,0,0,23 Oct 2011,v South Africa Gqeberha,2,11,1,1,1,1


In [112]:
train['stumpings'].unique()

array(['0', '2', '1', '-'], dtype=object)

In [40]:
cols = ['wickets', 'runs_conceded', 'catches', 'stumpings']
f = lambda x: 0 if x=='-' else x

for col in cols:
    train[col +'_1'] = train[col].map(f)   
    train[col +'_1']=train[col +'_1'].astype(int)  

In [41]:
train['match_date_1'] = train['match_date'].astype('datetime64[ns]')

In [42]:
train.head()

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id,runs_scored_1,out_notout,is_match_complete,bowled_in_match,batted_in_match,wickets_1,runs_conceded_1,catches_1,stumpings_1,match_date_1
0,1,Pat Cummins,DNB,3,28,0,0,19 Oct 2011,v South Africa Centurion,1,-99,2,1,1,0,3,28,0,0,2011-10-19
1,2,Steve Smith,DNB,-,-,0,0,19 Oct 2011,v South Africa Centurion,1,-99,2,1,0,0,0,0,0,0,2011-10-19
2,10,Mitch Marsh,8*,1,19,1,0,19 Oct 2011,v South Africa Centurion,1,8,1,1,1,1,1,19,1,0,2011-10-19
3,13,David Warner,20,-,-,0,0,19 Oct 2011,v South Africa Centurion,1,20,0,1,0,1,0,0,0,0,2011-10-19
4,1,Pat Cummins,11*,1,73,0,0,23 Oct 2011,v South Africa Gqeberha,2,11,1,1,1,1,1,73,0,0,2011-10-23


In [152]:
train['player_name'].unique()

array(['Pat Cummins', 'Steve Smith', 'Mitch Marsh', 'David Warner',
       'Glenn Maxwell', 'Mitchell Starc.', 'Josh Hazlewood',
       'Rohit Sharma', 'Virat Kohli', 'Mohammed Shami',
       'Ravichandran Ashwin', 'Marcus Stoinis', 'Travis Head',
       'Adam Zampa', 'Hardik Pandya', 'Jasprit Bumrah', 'Kuldeep Yadav',
       'Alex Carey', 'Ravindra Jadeja', 'KL Rahul', 'Marnus Labuschagne',
       'Shreyas Iyer', 'Shardul Thakur', 'Cameron Green', 'Josh Inglis',
       'Sean Abbott', 'Shubman Gill', 'Ishan Kishan', 'Suryakumar Yadav',
       'Mohammed Siraj'], dtype=object)

In [44]:
#train['player1'] = train['player1'].apply(lambda x: convert_name(x))
#train.player1.unique()
def convert_name(col):
    return (col.split()[0][0] +' '+ col.split()[1] )

train['player1'] = train.player_name
train['player1'] = train['player1'].apply(lambda x: convert_name(x))
train.player1.unique()

array(['P Cummins', 'S Smith', 'M Marsh', 'D Warner', 'G Maxwell',
       'M Starc.', 'J Hazlewood', 'R Sharma', 'V Kohli', 'M Shami',
       'R Ashwin', 'M Stoinis', 'T Head', 'A Zampa', 'H Pandya',
       'J Bumrah', 'K Yadav', 'A Carey', 'R Jadeja', 'K Rahul',
       'M Labuschagne', 'S Iyer', 'S Thakur', 'C Green', 'J Inglis',
       'S Abbott', 'S Gill', 'I Kishan', 'S Yadav', 'M Siraj'],
      dtype=object)

In [45]:
train.shape

(2575, 21)

In [49]:
train["year"]=pd.to_datetime(train["match_date_1"]).dt.year
train["month"]=pd.to_datetime(train["match_date_1"]).dt.month
train["date"]=pd.to_datetime(train["match_date_1"]).dt.date
train["week"]=pd.to_datetime(train["match_date_1"]).dt.week
train["day"]=pd.to_datetime(train["match_date_1"]).dt.day_name()
train.drop('date', axis=1, inplace=True)

C:\Users\Acer\AppData\Local\Temp\ipykernel_13556\3132414304.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  train["week"]=pd.to_datetime(train["match_date_1"]).dt.week


In [51]:
pd.set_option('display.max_columns', 100)
train.head(2)

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id,runs_scored_1,out_notout,is_match_complete,bowled_in_match,batted_in_match,wickets_1,runs_conceded_1,catches_1,stumpings_1,match_date_1,player1,year,month,week,day
0,1,Pat Cummins,DNB,3,28,0,0,19 Oct 2011,v South Africa Centurion,1,-99,2,1,1,0,3,28,0,0,2011-10-19,P Cummins,2011,10,42,Wednesday
1,2,Steve Smith,DNB,-,-,0,0,19 Oct 2011,v South Africa Centurion,1,-99,2,1,0,0,0,0,0,0,2011-10-19,S Smith,2011,10,42,Wednesday


In [52]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2575 entries, 0 to 2574
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   player_id          2575 non-null   int64         
 1   player_name        2575 non-null   object        
 2   runs_scored        2575 non-null   object        
 3   wickets            2575 non-null   object        
 4   runs_conceded      2575 non-null   object        
 5   catches            2575 non-null   object        
 6   stumpings          2575 non-null   object        
 7   match_date         2575 non-null   object        
 8   opposition         2575 non-null   object        
 9   match_id           2575 non-null   int64         
 10  runs_scored_1      2575 non-null   int32         
 11  out_notout         2575 non-null   int64         
 12  is_match_complete  2575 non-null   int64         
 13  bowled_in_match    2575 non-null   int64         
 14  batted_i

In [54]:
train.opposition.unique()

array(['v South Africa Centurion', 'v South Africa Gqeberha',
       'v South Africa Durban', 'v Ireland Belfast', "v England Lord's",
       'v South Africa Melbourne', 'v South Africa Sydney',
       'v England Sydney', 'v India Melbourne', 'v Australia Melbourne',
       'v England Hobart', 'v New Zealand Auckland', 'v Scotland Hobart',
       'v England Southampton', 'v England Manchester', 'v England Leeds',
       'v New Zealand Sydney', 'v New Zealand Canberra',
       'v New Zealand Melbourne', 'v Pakistan Brisbane',
       'v Pakistan Melbourne', 'v Pakistan Perth', 'v Pakistan Sydney',
       'v Pakistan Adelaide', 'v New Zealand Hamilton',
       'v New Zealand Birmingham', 'v Bangladesh The Oval',
       'v England Birmingham', 'v India Chennai', 'v Australia Chennai',
       'v India Eden Gardens', 'v Australia Eden Gardens',
       'v India Indore', 'v Australia Indore', 'v India Bengaluru',
       'v Australia Bengaluru', 'v India Nagpur', 'v Australia Nagpur',
       'v

### Finding the opposition country

In [83]:
# #country_list= pd.DataFrame(["india","sri lanka","bangladesh", 'australia', 'south africa', 'west indies', 'new zealand','afganistan','netherlands', 'pakistan'], columns=['name'])
# country_list= ["india","sri lanka","bangladesh", 'australia', 'south africa', 'west indies', 'new zealand','afganistan','netherlands', 'pakistan']
# found_in_whole_string =[]
# matching_countries = []
# # for key, value in train['opposition'].items():
# #     for text in value:
# #         for country in country_list:
# #                 if country in text:
# #                     matching_countries.append(country)

# for country in country_list:
#     if country in train.opposition:
#         matching_countries.append(country)
# train['opposition_country'] = matching_countries

# #from dataprep.clean import clean_country
# import dataprep.clean
# df2 = clean_country(train, "opposition")
# df2

In [175]:
match = ["india","sri lanka","bangladesh", 'australia', 'south africa', 'west indies', 'new zealand','afghanistan'
         ,'netherlands', 'pakistan','england','ireland', 'zimbabwe','scotland','u.a.e.','canada', 'kenya', 'hong kong', 'nepal']
df = train.copy()
df['opposition'] =df['opposition'].str.lower()
pattern = '|'.join(match)
import re
def pattern_searcher(search_str:str, search_list:str):

    search_obj = re.search(search_list, search_str)
    if search_obj :
        return_str = search_str[search_obj.start(): search_obj.end()]
    else:
        return_str = 'NA'
    return return_str

In [176]:
df['opposition_country'] = df['opposition'].apply(lambda x: pattern_searcher(search_str=x, search_list=pattern))
df

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id,runs_scored_1,out_notout,is_match_complete,bowled_in_match,batted_in_match,wickets_1,runs_conceded_1,catches_1,stumpings_1,match_date_1,player1,year,month,week,day,opposition_country,match_bat_encoded,match_bowl_encoded
0,1,Pat Cummins,DNB,3,28,0,0,19 Oct 2011,v south africa centurion,1,-99,2,1,1,0,3,28,0,0,2011-10-19,P Cummins,2011,10,42,Wednesday,south africa,1.750000,93.000000
1,2,Steve Smith,DNB,-,-,0,0,19 Oct 2011,v south africa centurion,1,-99,2,1,0,0,0,0,0,0,2011-10-19,S Smith,2011,10,42,Wednesday,south africa,8.062500,0.380952
2,10,Mitch Marsh,8*,1,19,1,0,19 Oct 2011,v south africa centurion,1,8,1,1,1,1,1,19,1,0,2011-10-19,M Marsh,2011,10,42,Wednesday,south africa,18.750000,4.266667
3,13,David Warner,20,-,-,0,0,19 Oct 2011,v south africa centurion,1,20,0,1,0,1,0,0,0,0,2011-10-19,D Warner,2011,10,42,Wednesday,south africa,74.000000,0.006711
4,1,Pat Cummins,11*,1,73,0,0,23 Oct 2011,v south africa gqeberha,2,11,1,1,1,1,1,73,0,0,2011-10-23,P Cummins,2011,10,42,Sunday,south africa,1.750000,93.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2570,22,Ravindra Jadeja,DNB,3,35,0,0,28 Oct 2009,v australia nagpur,553,-99,2,1,1,0,3,35,0,0,2009-10-28,R Jadeja,2009,10,44,Wednesday,australia,2.152542,22.250000
2571,22,Ravindra Jadeja,DNB,2,41,0,0,31 Oct 2009,v australia delhi,554,-99,2,1,1,0,2,41,0,0,2009-10-31,R Jadeja,2009,10,44,Saturday,australia,2.152542,22.250000
2572,22,Ravindra Jadeja,23,0,44,1,0,5 Nov 2009,v australia hyderabad,555,23,0,1,1,1,0,44,1,0,2009-11-05,R Jadeja,2009,11,45,Thursday,australia,2.152542,22.250000
2573,22,Ravindra Jadeja,57,0,36,0,0,8 Nov 2009,v australia guwahati,556,57,0,1,1,1,0,36,0,0,2009-11-08,R Jadeja,2009,11,45,Sunday,australia,2.152542,22.250000


In [171]:
df.isna().sum().sum()

0

In [177]:
#(df['opposition_country']=='NA').sum()
#df[df['opposition_country']=='NA']



,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id,runs_scored_1,out_notout,is_match_complete,bowled_in_match,batted_in_match,wickets_1,runs_conceded_1,catches_1,stumpings_1,match_date_1,player1,year,month,week,day,opposition_country,match_bat_encoded,match_bowl_encoded


In [85]:
#prob_df=df.groupby(['Cabin'])['Survived'].mean()

prob_df_batting = df.groupby(['player_name'])['batted_in_match'].mean()
prob_df_bowling = df.groupby(['player_name'])['bowled_in_match'].mean()

prob_df_batting=pd.DataFrame(prob_df_batting)
prob_df_bowling=pd.DataFrame(prob_df_bowling)


In [418]:
prob_df_batting['not_batted_in_match']=1-prob_df_batting['batted_in_match']
prob_df_bowling['not_bowled_in_match']=1-prob_df_bowling['bowled_in_match']

prob_df_batting['Probability_ratio_batting']=prob_df_batting['batted_in_match']/prob_df_batting['not_batted_in_match']
prob_df_bowling['Probability_ratio_bowling']=prob_df_bowling['bowled_in_match']/prob_df_bowling['not_bowled_in_match']

probability_encoded_bat=prob_df_batting['Probability_ratio_batting'].to_dict()
probability_encoded_bowl=prob_df_bowling['Probability_ratio_bowling'].to_dict()

df['match_bat_encoded']=df['player_name'].map(probability_encoded_bat)
df['match_bowl_encoded']=df['player_name'].map(probability_encoded_bowl)

In [419]:
df.head(10)

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id,runs_scored_1,out_notout,is_match_complete,bowled_in_match,batted_in_match,wickets_1,runs_conceded_1,catches_1,stumpings_1,match_date_1,player1,year,month,week,day,opposition_country,match_bat_encoded,match_bowl_encoded,day_of_the_week_code
0,1,Pat Cummins,DNB,3,28,0,0,19 Oct 2011,v south africa centurion,1,-99,2,1,1,0,3,28,0,0,2011-10-19,P Cummins,2011,10,42,Wednesday,south africa,1.7500,inf,3
1,2,Steve Smith,DNB,-,-,0,0,19 Oct 2011,v south africa centurion,1,-99,2,1,0,0,0,0,0,0,2011-10-19,S Smith,2011,10,42,Wednesday,south africa,8.0625,0.380952,3
2,10,Mitch Marsh,8*,1,19,1,0,19 Oct 2011,v south africa centurion,1,8,1,1,1,1,1,19,1,0,2011-10-19,M Marsh,2011,10,42,Wednesday,south africa,18.7500,4.266667,3
3,13,David Warner,20,-,-,0,0,19 Oct 2011,v south africa centurion,1,20,0,1,0,1,0,0,0,0,2011-10-19,D Warner,2011,10,42,Wednesday,south africa,74.0000,0.006711,3
4,1,Pat Cummins,11*,1,73,0,0,23 Oct 2011,v south africa gqeberha,2,11,1,1,1,1,1,73,0,0,2011-10-23,P Cummins,2011,10,42,Sunday,south africa,1.7500,inf,7
5,2,Steve Smith,26,1,24,1,0,23 Oct 2011,v south africa gqeberha,2,26,0,1,1,1,1,24,1,0,2011-10-23,S Smith,2011,10,42,Sunday,south africa,8.0625,0.380952,7
6,13,David Warner,74,-,-,0,0,23 Oct 2011,v south africa gqeberha,2,74,0,1,0,1,0,0,0,0,2011-10-23,D Warner,2011,10,42,Sunday,south africa,74.0000,0.006711,7
7,1,Pat Cummins,6*,1,49,0,0,28 Oct 2011,v south africa durban,3,6,1,1,1,1,1,49,0,0,2011-10-28,P Cummins,2011,10,43,Friday,south africa,1.7500,inf,5
8,13,David Warner,10,-,-,1,0,28 Oct 2011,v south africa durban,3,10,0,1,0,1,0,0,1,0,2011-10-28,D Warner,2011,10,43,Friday,south africa,74.0000,0.006711,5
9,1,Pat Cummins,TDNB,1,11,0,0,23 Jun 2012,v ireland belfast,4,-99,2,0,1,0,1,11,0,0,2012-06-23,P Cummins,2012,6,25,Saturday,ireland,1.7500,inf,6


In [420]:
df['match_bat_encoded'].replace([np.inf, -np.inf], df.loc[df['match_bat_encoded'] != np.inf, 'match_bat_encoded'].max(), inplace=True)  ## replacing infity with max 
df['match_bowl_encoded'].replace([np.inf, -np.inf], df.loc[df['match_bowl_encoded'] != np.inf, 'match_bowl_encoded'].max(), inplace=True)

In [181]:
df.groupby(['player_id', 'player1'])['match_bowl_encoded'].mean()

player_id  player1      
1          P Cummins        93.000000
2          S Smith           0.380952
3          A Carey           0.000000
4          J Inglis          0.000000
5          S Abbott         93.000000
6          M Labuschagne     0.357143
7          C Green          93.000000
8          J Hazlewood      73.000000
9          T Head            1.230769
10         M Marsh           4.266667
11         G Maxwell         3.777778
12         M Stoinis         8.142857
13         D Warner          0.006711
14         A Zampa          93.000000
15         M Starc.         93.000000
16         R Sharma          0.178404
17         H Pandya         12.666667
18         S Gill            0.000000
19         V Kohli           0.206009
20         S Iyer            0.119048
21         K Rahul           0.000000
22         R Jadeja         22.250000
23         S Thakur         43.000000
24         J Bumrah         77.000000
25         M Siraj          29.000000
26         K Yadav       

In [135]:
team_bat_aus_stat = pd.read_csv('team_aus_batting_stat.csv', header=0)
team_bat_aus_stat.head()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,SR,100,50,0s
0,SA Abbott,2014-2023,17,14,1,228,54,17.53,107.54,-,1,1
1,AC Agar,2015-2023,22,18,5,322,48*,24.76,82.98,-,-,3
2,JP Behrendorff,2019-2022,12,6,3,21,11*,7.00,91.30,-,-,1
3,AT Carey,2018-2023,71,65,11,1814,106,33.59,88.48,1,8,3
4,PJ Cummins,2011-2023,77,49,17,364,36,11.37,77.61,-,-,8


In [136]:
def convert_name(col):
    return (col.split()[0][0] +' '+ col.split()[1] )

team_bat_aus_stat['Player1'] = team_bat_aus_stat.Player
team_bat_aus_stat['Player1'] = team_bat_aus_stat['Player1'].apply(lambda x: convert_name(x))
team_bat_aus_stat['Player1'].unique()

array(['S Abbott', 'A Agar', 'J Behrendorff', 'A Carey', 'P Cummins',
       'T David', 'N Ellis', 'A Finch', 'C Green', 'A Hardie',
       'J Hazlewood', 'T Head', 'J Inglis', 'S Johnson', 'M Kuhnemann',
       'M Labuschagne', 'B McDermott', 'M Marsh', 'G Maxwell', 'M Neser',
       'J Richardson', 'T Sangha', 'M Short', 'S Smith', 'M Starc',
       'M Stoinis', 'M Swepson', 'D Warner', 'A Zampa'], dtype=object)

In [115]:
df['player1'].unique()

array(['P Cummins', 'S Smith', 'M Marsh', 'D Warner', 'G Maxwell',
       'M Starc.', 'J Hazlewood', 'R Sharma', 'V Kohli', 'M Shami',
       'R Ashwin', 'M Stoinis', 'T Head', 'A Zampa', 'H Pandya',
       'J Bumrah', 'K Yadav', 'A Carey', 'R Jadeja', 'K Rahul',
       'M Labuschagne', 'S Iyer', 'S Thakur', 'C Green', 'J Inglis',
       'S Abbott', 'S Gill', 'I Kishan', 'S Yadav', 'M Siraj'],
      dtype=object)

In [116]:
team_bat_ind_stat = pd.read_csv('team_india_batting_stat.csv', header=0)
team_bat_ind_stat.head()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,SR,100,50,0s
0,Arshdeep Singh,2022-2022,3,1,0,9,9,9.00,100.00,-,-,-
1,R Ashwin,2010-2023,115,63,20,707,65,16.44,86.96,-,1,6
2,Avesh Khan,2022-2022,5,2,0,13,10,6.50,72.22,-,-,-
3,JJ Bumrah,2016-2023,78,23,13,73,16,7.30,56.15,-,-,5
4,YS Chahal,2016-2023,72,14,5,77,18*,8.55,54.60,-,-,1


In [117]:
team_bat_ind_stat['Player1'] = team_bat_ind_stat.Player
team_bat_ind_stat['Player1'] = team_bat_ind_stat['Player1'].apply(lambda x: convert_name(x))
team_bat_ind_stat['Player1'].unique()

array(['A Singh', 'R Ashwin', 'A Khan', 'J Bumrah', 'Y Chahal',
       'D Chahar', 'S Dhawan', 'R Gaikwad', 'D Hooda', 'I Kishan',
       'S Iyer', 'V Iyer', 'R Jadeja', 'V Kohli', 'K Yadav', 'B Kumar',
       'M Shami', 'M Siraj', 'M Kumar', 'H Pandya', 'R Pant', 'A Patel',
       'M Prasidh', 'K Rahul', 'R Bishnoi', 'S Ahmed', 'S Samson',
       'K Sen', 'R Sharma', 'S Gill', 'S Thakur', 'N Tilak', 'U Malik',
       'J Unadkat', 'W Sundar', 'J Yadav', 'S Yadav'], dtype=object)

In [125]:
#df.groupby('player1')['player_id'].mean().astype(int)
list = df['player1'].unique()
list

array(['P Cummins', 'S Smith', 'M Marsh', 'D Warner', 'G Maxwell',
       'M Starc.', 'J Hazlewood', 'R Sharma', 'V Kohli', 'M Shami',
       'R Ashwin', 'M Stoinis', 'T Head', 'A Zampa', 'H Pandya',
       'J Bumrah', 'K Yadav', 'A Carey', 'R Jadeja', 'K Rahul',
       'M Labuschagne', 'S Iyer', 'S Thakur', 'C Green', 'J Inglis',
       'S Abbott', 'S Gill', 'I Kishan', 'S Yadav', 'M Siraj'],
      dtype=object)

In [124]:
team_bat_ind_stat

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,SR,100,50,0s,Player1
0,Arshdeep Singh,2022-2022,3,1,0,9,9,9.00,100.00,-,-,-,A Singh
1,R Ashwin,2010-2023,115,63,20,707,65,16.44,86.96,-,1,6,R Ashwin
2,Avesh Khan,2022-2022,5,2,0,13,10,6.50,72.22,-,-,-,A Khan
3,JJ Bumrah,2016-2023,78,23,13,73,16,7.30,56.15,-,-,5,J Bumrah
4,YS Chahal,2016-2023,72,14,5,77,18*,8.55,54.60,-,-,1,Y Chahal
5,DL Chahar,2018-2022,13,9,3,203,69*,33.83,98.06,-,2,1,D Chahar
6,S Dhawan,2010-2022,167,164,10,6793,143,44.11,91.35,17,39,5,S Dhawan
7,RD Gaikwad,2022-2023,4,4,0,106,71,26.50,73.10,-,1,-,R Gaikwad
8,DJ Hooda,2022-2022,10,7,1,153,33,25.50,80.95,-,-,-,D Hooda
9,Ishan Kishan,2021-2023,25,22,2,886,210,44.30,102.42,1,7,-,I Kishan


In [129]:
team_bat_ind_stat['wc_team_player'] = team_bat_ind_stat.Player1.isin(list)
#team_bat_ind_stat


In [130]:
team_bat_ind_stat = team_bat_ind_stat[team_bat_ind_stat['wc_team_player'] ==True]
team_bat_ind_stat

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,SR,100,50,0s,Player1,wc_team_player
1,R Ashwin,2010-2023,115,63,20,707,65,16.44,86.96,-,1,6,R Ashwin,True
3,JJ Bumrah,2016-2023,78,23,13,73,16,7.30,56.15,-,-,5,J Bumrah,True
9,Ishan Kishan,2021-2023,25,22,2,886,210,44.30,102.42,1,7,-,I Kishan,True
10,SS Iyer,2017-2023,47,42,3,1801,113*,46.17,97.88,3,14,-,S Iyer,True
12,RA Jadeja,2009-2023,186,127,45,2636,87,32.14,84.43,-,13,6,R Jadeja,True
13,V Kohli,2008-2023,281,269,41,13083,183,57.38,93.78,47,66,15,V Kohli,True
14,Kuldeep Yadav,2017-2023,90,34,17,170,19,10.00,56.29,-,-,3,K Yadav,True
16,Mohammed Shami,2013-2023,94,44,19,210,25,8.40,85.71,-,-,6,M Shami,True
17,Mohammed Siraj,2019-2023,30,13,7,37,9,6.16,40.21,-,-,1,M Siraj,True
19,HH Pandya,2016-2023,82,60,8,1758,92*,33.80,110.21,-,11,4,H Pandya,True


In [137]:
new_item = 'M Starc'

list = [new_item if x == 'M Starc.' else x for x in list]

team_bat_aus_stat['wc_team_player'] = team_bat_aus_stat.Player1.isin(list)
team_bat_aus_stat = team_bat_aus_stat[team_bat_aus_stat['wc_team_player'] ==True]
team_bat_aus_stat

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,SR,100,50,0s,Player1,wc_team_player
0,SA Abbott,2014-2023,17,14,1,228,54,17.53,107.54,-,1,1,S Abbott,True
3,AT Carey,2018-2023,71,65,11,1814,106,33.59,88.48,1,8,3,A Carey,True
4,PJ Cummins,2011-2023,77,49,17,364,36,11.37,77.61,-,-,8,P Cummins,True
8,C Green,2020-2023,20,17,7,379,89*,37.90,84.78,-,1,-,C Green,True
10,JR Hazlewood,2010-2023,74,26,21,117,23*,23.40,91.40,-,-,2,J Hazlewood,True
11,TM Head,2016-2023,58,55,5,2064,152,41.28,99.51,3,15,2,T Head,True
12,JP Inglis,2022-2023,8,8,0,143,50,17.87,93.46,-,1,1,J Inglis,True
15,M Labuschagne,2020-2023,38,36,2,1268,124,37.29,87.50,2,8,2,M Labuschagne,True
17,MR Marsh,2011-2023,79,75,10,2231,102*,34.32,94.21,1,17,5,M Marsh,True
18,GJ Maxwell,2012-2023,129,118,14,3495,108,33.60,124.68,2,23,9,G Maxwell,True


In [142]:
team_bat_aus_ind_stat = pd.concat([team_bat_aus_stat, team_bat_ind_stat], ignore_index=True, sort=True)
#team_bat_aus_ind_stat

In [148]:
team_bat_aus_ind_stat = team_bat_aus_ind_stat.replace({'-':'0'})
team_bat_aus_ind_stat['HS'] =team_bat_aus_ind_stat['HS'].str.replace('*','')
#team_bat_aus_ind_stat

C:\Users\Acer\AppData\Local\Temp\ipykernel_13556\4265709782.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  team_bat_aus_ind_stat['HS'] =team_bat_aus_ind_stat['HS'].str.replace('*','')


In [152]:
team_bat_aus_ind_stat.dtypes

0s                 object
100                object
50                 object
Ave                object
HS                 object
Inns               object
Mat                 int64
NO                  int64
Player             object
Player1            object
Runs               object
SR                float64
Span               object
wc_team_player       bool
dtype: object

In [151]:
team_bat_aus_ind_stat.columns

Index(['0s', '100', '50', 'Ave', 'HS', 'Inns', 'Mat', 'NO', 'Player',
       'Player1', 'Runs', 'SR', 'Span', 'wc_team_player'],
      dtype='object')

In [155]:
team_bat_aus_ind_stat[[ 'Ave',  ]] = team_bat_aus_ind_stat[[ 'Ave',]].astype(float)
team_bat_aus_ind_stat[['0s', '100', '50', 'HS', 'Inns','Runs']] = team_bat_aus_ind_stat[['0s', '100', '50', 'HS', 'Inns','Runs']].astype(int)
team_bat_aus_ind_stat.dtypes

0s                  int32
100                 int32
50                  int32
Ave               float64
HS                  int32
Inns                int32
Mat                 int64
NO                  int64
Player             object
Player1            object
Runs                int32
SR                float64
Span               object
wc_team_player       bool
dtype: object

In [182]:
team_bat_aus_ind_stat

,0s,100,50,Ave,HS,Inns,Mat,NO,Player,Player1,Runs,SR,Span,wc_team_player
0,1,0,1,17.53,54,14,17,1,SA Abbott,S Abbott,228,107.54,2014-2023,True
1,3,1,8,33.59,106,65,71,11,AT Carey,A Carey,1814,88.48,2018-2023,True
2,8,0,0,11.37,36,49,77,17,PJ Cummins,P Cummins,364,77.61,2011-2023,True
3,0,0,1,37.90,89,17,20,7,C Green,C Green,379,84.78,2020-2023,True
4,2,0,0,23.40,23,26,74,21,JR Hazlewood,J Hazlewood,117,91.40,2010-2023,True
5,2,3,15,41.28,152,55,58,5,TM Head,T Head,2064,99.51,2016-2023,True
6,1,0,1,17.87,50,8,8,0,JP Inglis,J Inglis,143,93.46,2022-2023,True
7,2,2,8,37.29,124,36,38,2,M Labuschagne,M Labuschagne,1268,87.50,2020-2023,True
8,5,1,17,34.32,102,75,79,10,MR Marsh,M Marsh,2231,94.21,2011-2023,True
9,9,2,23,33.60,108,118,129,14,GJ Maxwell,G Maxwell,3495,124.68,2012-2023,True


In [239]:
df.day.unique()

array(['Wednesday', 'Sunday', 'Friday', 'Saturday', 'Thursday', 'Tuesday',
       'Monday'], dtype=object)

In [478]:
col_to_merge = ['0s', '100', '50', 'Ave', 'HS', 'Inns', 'Mat', 'NO',  'SR' ,'Player1']
dow_dict = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}
df['player1'] =df['player1'].str.replace('.','')
df['day_of_the_week_code'] = df.day.map(dow_dict)
df_merged = pd.merge(df,
                     team_bat_aus_ind_stat[col_to_merge],left_on='player1',right_on='Player1',how='left')



C:\Users\Acer\AppData\Local\Temp\ipykernel_13556\464648975.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['player1'] =df['player1'].str.replace('.','')


In [479]:
df_merged.isna().sum().sum()

0

In [480]:
print(df.shape, df_merged.shape)

(2575, 29) (2575, 39)


In [481]:
df.columns

Index(['player_id', 'player_name', 'runs_scored', 'wickets', 'runs_conceded',
       'catches', 'stumpings', 'match_date', 'opposition', 'match_id',
       'runs_scored_1', 'out_notout', 'is_match_complete', 'bowled_in_match',
       'batted_in_match', 'wickets_1', 'runs_conceded_1', 'catches_1',
       'stumpings_1', 'match_date_1', 'player1', 'year', 'month', 'week',
       'day', 'opposition_country', 'match_bat_encoded', 'match_bowl_encoded',
       'day_of_the_week_code'],
      dtype='object')

In [482]:
df_merged1 = pd.merge(df,
                     team_bat_aus_ind_stat[col_to_merge],left_on='player1',right_on='Player1',how='left')
#df_merged1.to_csv('model_data_bat_stat_merged.csv', index=False)

In [165]:
# categorical encoding columns -> 'opposition_country', 
# target column -> 'wickets_1',  'runs_scored_1'
# columns to drop -> 'player_id', 'player_name', 'runs_scored', 'wickets', 'runs_conceded',
#       'catches', 'stumpings', 'match_date', 'opposition', 'match_id','runs_scored_1', 'match_date_1''player1','Player1'
# submission data madeup

In [484]:
df_merged1.columns

Index(['player_id', 'player_name', 'runs_scored', 'wickets', 'runs_conceded',
       'catches', 'stumpings', 'match_date', 'opposition', 'match_id',
       'runs_scored_1', 'out_notout', 'is_match_complete', 'bowled_in_match',
       'batted_in_match', 'wickets_1', 'runs_conceded_1', 'catches_1',
       'stumpings_1', 'match_date_1', 'player1', 'year', 'month', 'week',
       'day', 'opposition_country', 'match_bat_encoded', 'match_bowl_encoded',
       'day_of_the_week_code', '0s', '100', '50', 'Ave', 'HS', 'Inns', 'Mat',
       'NO', 'SR', 'Player1'],
      dtype='object')

(30, 3)

In [423]:
df_merged1.columns

Index(['player_id', 'player_name', 'runs_scored', 'wickets', 'runs_conceded',
       'catches', 'stumpings', 'match_date', 'opposition', 'match_id',
       'runs_scored_1', 'out_notout', 'is_match_complete', 'bowled_in_match',
       'batted_in_match', 'wickets_1', 'runs_conceded_1', 'catches_1',
       'stumpings_1', 'match_date_1', 'player1', 'year', 'month', 'week',
       'day', 'match_bat_encoded', 'match_bowl_encoded',
       'day_of_the_week_code', '0s', '100', '50', 'Ave', 'HS', 'Inns', 'Mat',
       'NO', 'SR', 'Player1', 'avg_runs_scored_1_encode',
       'avg_out_notout_encode', 'avg_batted_in_match_encode',
       'avg_bowled_in_match_encode', 'avg_wickets_1_encode',
       'avg_runs_conceded_1_encode', 'avg_catches_1_encode',
       'avg_stumpings_1_encode', 'opposition_country_afghanistan',
       'opposition_country_australia', 'opposition_country_bangladesh',
       'opposition_country_canada', 'opposition_country_england',
       'opposition_country_hong kong', 'opposi

In [299]:
#df[df['year']>2020].player_name.nunique()
#df[df['year']>2020].groupby(['player_name'])['batted_in_match'].mean()

In [300]:
avg_df_batting = df[df['year']>2020].groupby(['player_name'])['runs_scored_1'].mean()
no_df_batting = df[df['year']>2020].groupby(['player_name'])['out_notout'].mean()
match_df_batting = df[df['year']>2020].groupby(['player_name'])['batted_in_match'].mean()
match_df_bowling = df[df['year']>2020].groupby(['player_name'])['bowled_in_match'].mean()
wkt_df_bowling =  df[df['year']>2020].groupby(['player_name'])['wickets_1'].mean()
avgrun_df_bowling =  df[df['year']>2020].groupby(['player_name'])['runs_conceded_1'].mean()
catch_df_bowling =  df[df['year']>2020].groupby(['player_name'])['catches_1'].mean()
stump_df_bowling =  df[df['year']>2020].groupby(['player_name'])['stumpings_1'].mean()


avg_df_batting=pd.DataFrame(avg_df_batting)
no_df_batting=pd.DataFrame(no_df_batting)
match_df_batting=pd.DataFrame(match_df_batting)
match_df_bowling=pd.DataFrame(match_df_bowling)
wkt_df_bowling=pd.DataFrame(wkt_df_bowling)
avgrun_df_bowling=pd.DataFrame(avgrun_df_bowling)
catch_df_bowling=pd.DataFrame(catch_df_bowling)
stump_df_bowling=pd.DataFrame(stump_df_bowling)

avg_df_batting_encode=avg_df_batting['runs_scored_1'].to_dict()
no_df_batting_encode=no_df_batting['out_notout'].to_dict()
match_df_batting_encode=match_df_batting['batted_in_match'].to_dict()
match_df_bowling_encode=match_df_bowling['bowled_in_match'].to_dict()
wkt_df_bowling_encode=wkt_df_bowling['wickets_1'].to_dict()
avgrun_df_bowling_encode=avgrun_df_bowling['runs_conceded_1'].to_dict()
catch_df_bowling_encode=catch_df_bowling['catches_1'].to_dict()
stump_df_bowling_encode=stump_df_bowling['stumpings_1'].to_dict()

df_merged1['avg_df_batting_encode']=df_merged1['player_name'].map(probability_encoded_bat)
df_merged1['no_df_batting_encode']=df_merged1['player_name'].map(probability_encoded_bat)
df_merged1['match_df_batting_encode']=df_merged1['player_name'].map(probability_encoded_bat)
df_merged1['match_df_bowling_encode']=df_merged1['player_name'].map(probability_encoded_bat)
df_merged1['wkt_df_bowling_encode']=df_merged1['player_name'].map(probability_encoded_bat)
df_merged1['avgrun_df_bowling_encode']=df_merged1['player_name'].map(probability_encoded_bat)
df_merged1['catch_df_bowling_encode']=df_merged1['player_name'].map(probability_encoded_bat)
df_merged1['stump_df_bowling_encode']=df_merged1['player_name'].map(probability_encoded_bat)

#df['match_bat_encoded'].replace([np.inf, -np.inf], df.loc[df['match_bat_encoded'] != np.inf, 'match_bat_encoded'].max(), inplace=True)  ## replacing infity with max 
#df['match_bowl_encoded'].replace([np.inf, -np.inf], df.loc[df['match_bowl_encoded'] != np.inf, 'match_bowl_encoded'].max(), inplace=True)

In [424]:
df_merged1['match_bowl_encoded']=df_merged1['player_name'].map(probability_encoded_bowl)
df_merged1['match_bowl_encoded'].replace([np.inf, -np.inf], df_merged1.loc[df_merged1['match_bowl_encoded'] != np.inf, 'match_bowl_encoded'].max(), inplace=True)

In [328]:
cols_to_drop = ['avg_df_batting_encode','no_df_batting_encode','match_df_batting_encode','match_df_bowling_encode','wkt_df_bowling_encode',
                'avgrun_df_bowling_encode','catch_df_bowling_encode','stump_df_bowling_encode']
df_merged1.drop(cols_to_drop, axis=1, inplace=True)

In [323]:
def f(data, col1,col2):
    temp = df_merged1[df_merged1['year']>2020].groupby(col1)[col2].mean()
    temp=pd.DataFrame(temp)
    temp_encode=temp[col2].to_dict()
    data['avg_'+col2+'_encode']=data[col1].map(temp_encode)
    data['avg_'+col2+'_encode'].replace([np.inf, -np.inf],0, inplace=True)

In [485]:
f(df_merged1, 'player_id', 'runs_scored_1')
f(df_merged1, 'player_id', 'out_notout')
f(df_merged1, 'player_id', 'batted_in_match')
f(df_merged1, 'player_id', 'bowled_in_match')
f(df_merged1, 'player_id', 'wickets_1')
f(df_merged1, 'player_id', 'runs_conceded_1')
f(df_merged1, 'player_id', 'catches_1')
f(df_merged1, 'player_id', 'stumpings_1')

In [486]:

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
x=df_merged1.select_dtypes(include=numerics)
x.isin([np.inf, -np.inf]).sum().sum()

0

In [487]:
df_merged1.columns

Index(['player_id', 'player_name', 'runs_scored', 'wickets', 'runs_conceded',
       'catches', 'stumpings', 'match_date', 'opposition', 'match_id',
       'runs_scored_1', 'out_notout', 'is_match_complete', 'bowled_in_match',
       'batted_in_match', 'wickets_1', 'runs_conceded_1', 'catches_1',
       'stumpings_1', 'match_date_1', 'player1', 'year', 'month', 'week',
       'day', 'opposition_country', 'match_bat_encoded', 'match_bowl_encoded',
       'day_of_the_week_code', '0s', '100', '50', 'Ave', 'HS', 'Inns', 'Mat',
       'NO', 'SR', 'Player1', 'avg_runs_scored_1_encode',
       'avg_out_notout_encode', 'avg_batted_in_match_encode',
       'avg_bowled_in_match_encode', 'avg_wickets_1_encode',
       'avg_runs_conceded_1_encode', 'avg_catches_1_encode',
       'avg_stumpings_1_encode'],
      dtype='object')

In [373]:
submission_data.head()

,player_id,runs,wickets,Player1,0s,100,50,Ave,HS,Inns,Mat,NO,SR,avg_runs_scored_1_encode,avg_out_notout_encode,avg_batted_in_match_encode,avg_bowled_in_match_encode,avg_wickets_1_encode,avg_runs_conceded_1_encode,avg_catches_1_encode,avg_stumpings_1_encode
0,1,30,2,P Cummins,8,0,0,11.37,36,49,77,17,77.61,-27.250000,1.000000,0.625000,1.0,1.875,41.500000,0.500000,0.000000
1,2,24,0,S Smith,7,12,30,44.33,164,129,145,15,87.66,33.941176,0.294118,0.941176,0.0,0.000,0.000000,0.705882,0.000000
2,3,45,1,A Carey,3,1,8,33.59,106,65,71,11,88.48,21.517241,0.206897,0.965517,0.0,0.000,0.000000,1.137931,0.103448
3,4,5,2,J Inglis,1,0,1,17.87,50,8,8,0,93.46,17.875000,0.000000,1.000000,0.0,0.000,0.000000,0.250000,0.000000
4,5,9,0,S Abbott,1,0,1,17.53,54,14,17,1,107.54,-5.066667,0.466667,0.800000,1.0,1.200,41.866667,0.466667,0.000000


In [526]:
submission_data = pd.read_csv('sample_submission_Gk1Mne1.csv')
submission_data.shape
temp = pd.DataFrame(df_merged1.groupby(['Player1','player_name'])['player_id'].mean())
temp = temp.reset_index()
temp['player_id'] = temp['player_id'].astype(int)
submission_data = pd.merge(submission_data, temp, left_on='player_id', right_on='player_id', how='left' )

submission_data = pd.merge(submission_data,
                      team_bat_aus_ind_stat[['Player1','0s','100', '50', 'Ave', 'HS', 'Inns', 'Mat', 'NO', 'SR']],
                      left_on='Player1',right_on='Player1',how='left')

f(submission_data, 'player_id', 'runs_scored_1')
f(submission_data, 'player_id', 'out_notout')
f(submission_data, 'player_id', 'batted_in_match')
f(submission_data, 'player_id', 'bowled_in_match')
f(submission_data, 'player_id', 'wickets_1')
f(submission_data, 'player_id', 'runs_conceded_1')
f(submission_data, 'player_id', 'catches_1')
f(submission_data, 'player_id', 'stumpings_1')

submission_data['match_bat_encoded']=submission_data['player_name'].map(probability_encoded_bat)
submission_data['match_bowl_encoded']=submission_data['player_name'].map(probability_encoded_bowl)
submission_data['match_bat_encoded'].replace([np.inf, -np.inf], df.loc[df['match_bat_encoded'] != np.inf, 'match_bat_encoded'].max(), inplace=True)  ## replacing infity with max 
submission_data['match_bowl_encoded'].replace([np.inf, -np.inf], df.loc[df['match_bowl_encoded'] != np.inf, 'match_bowl_encoded'].max(), inplace=True)

col_to_add = ['opposition_country_afghanistan', 'opposition_country_australia','opposition_country_bangladesh', 'opposition_country_canada',
       'opposition_country_england', 'opposition_country_hong kong','opposition_country_india', 'opposition_country_ireland',
       'opposition_country_kenya', 'opposition_country_nepal','opposition_country_netherlands', 'opposition_country_new zealand',
       'opposition_country_pakistan', 'opposition_country_scotland','opposition_country_south africa', 'opposition_country_sri lanka',
       'opposition_country_u.a.e.', 'opposition_country_west indies','opposition_country_zimbabwe']

for col in col_to_add:
    submission_data[col]=0

submission_data['opposition_country_india'] = np.where(submission_data['player_id']>15,0,1)
submission_data['opposition_country_australia'] = np.where(submission_data['player_id']>15,1,0)

submission_data['match_date_1']='2023-10-08'
submission_data['match_date_1'] =submission_data['match_date_1'].astype('datetime64[ns]')
submission_data["year"]=pd.to_datetime(submission_data["match_date_1"]).dt.year
submission_data["month"]=pd.to_datetime(submission_data["match_date_1"]).dt.month
submission_data["date"]=pd.to_datetime(submission_data["match_date_1"]).dt.date
submission_data["week"]=pd.to_datetime(submission_data["match_date_1"]).dt.week
submission_data["day"]=pd.to_datetime(submission_data["match_date_1"]).dt.day_name()
dow_dict = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}
submission_data['day_of_the_week_code'] = submission_data.day.map(dow_dict)
submission_data['is_match_complete']=1
submission_data.drop(['match_date_1','day'], axis=1, inplace=True)

C:\Users\Acer\AppData\Local\Temp\ipykernel_13556\164484603.py:43: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  submission_data["week"]=pd.to_datetime(submission_data["match_date_1"]).dt.week


In [515]:
submission_data.columns

Index(['player_id', 'runs', 'wickets', 'Player1', 'player_name', '0s', '100',
       '50', 'Ave', 'HS', 'Inns', 'Mat', 'NO', 'SR',
       'avg_runs_scored_1_encode', 'avg_out_notout_encode',
       'avg_batted_in_match_encode', 'avg_bowled_in_match_encode',
       'avg_wickets_1_encode', 'avg_runs_conceded_1_encode',
       'avg_catches_1_encode', 'avg_stumpings_1_encode', 'match_bat_encoded',
       'match_bowl_encoded', 'opposition_country_afghanistan',
       'opposition_country_australia', 'opposition_country_bangladesh',
       'opposition_country_canada', 'opposition_country_england',
       'opposition_country_hong kong', 'opposition_country_india',
       'opposition_country_ireland', 'opposition_country_kenya',
       'opposition_country_nepal', 'opposition_country_netherlands',
       'opposition_country_new zealand', 'opposition_country_pakistan',
       'opposition_country_scotland', 'opposition_country_south africa',
       'opposition_country_sri lanka', 'opposition_count

In [449]:
submission_data.isna().sum().sum()

0

In [334]:
# col_to_merge= ['player_id', 'match_bat_encoded', 'match_bowl_encoded', '0s','100', '50', 'Ave', 'HS', 'Inns', 'Mat', 'NO', 'SR',
#                'avg_runs_scored_1_encode','avg_out_notout_encode', 'avg_batted_in_match_encode','avg_bowled_in_match_encode', 
#                'avg_wickets_1_encode','avg_runs_conceded_1_encode', 'avg_catches_1_encode','avg_stumpings_1_encode']
# submission_data = pd.merge(submission_data,
#                      df_merged1[col_to_merge],left_on='player_id',right_on='player_id',how='inner')

# 'is_match_complete', 'bowled_in_match', 'batted_in_match', 'runs_conceded_1', 'catches_1', 'stumpings_1',
# 'year', 'month', 'week',  'day_of_the_week_code'

In [489]:
df_merged1.columns

Index(['player_id', 'player_name', 'runs_scored', 'wickets', 'runs_conceded',
       'catches', 'stumpings', 'match_date', 'opposition', 'match_id',
       'runs_scored_1', 'out_notout', 'is_match_complete', 'bowled_in_match',
       'batted_in_match', 'wickets_1', 'runs_conceded_1', 'catches_1',
       'stumpings_1', 'match_date_1', 'player1', 'year', 'month', 'week',
       'day', 'match_bat_encoded', 'match_bowl_encoded',
       'day_of_the_week_code', '0s', '100', '50', 'Ave', 'HS', 'Inns', 'Mat',
       'NO', 'SR', 'Player1', 'avg_runs_scored_1_encode',
       'avg_out_notout_encode', 'avg_batted_in_match_encode',
       'avg_bowled_in_match_encode', 'avg_wickets_1_encode',
       'avg_runs_conceded_1_encode', 'avg_catches_1_encode',
       'avg_stumpings_1_encode', 'opposition_country_afghanistan',
       'opposition_country_australia', 'opposition_country_bangladesh',
       'opposition_country_canada', 'opposition_country_england',
       'opposition_country_hong kong', 'opposi

In [488]:
#df_merged = pd.get_dummies(df_merged, columns=['opposition_country'])
df_merged1 = pd.get_dummies(df_merged1, columns=['opposition_country'])

In [630]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

stat_ind_vs_aus.select_dtypes(include=numerics).columns.tolist()

['Mat',
 'NO',
 'HS',
 'Ave',
 'SR',
 '0s',
 'From',
 'To',
 'bat_prob',
 'match_per_year',
 '100_rank',
 '50_rank',
 'Inns_rank',
 'Inns_bowl',
 'Mdns_bowl',
 'Ave_bowl',
 'Econ_bowl',
 '5_bowl',
 '10_bowl',
 'Ct_bowl',
 'St_bowl',
 'BBI_wkts_bowl',
 'BBI_runs_bowl',
 'To_bowl',
 'bowl_prob_bowl',
 'Mdns_rank_bowl',
 'BBI_wkts_rank_bowl',
 'performance_rank_bowl']

In [ ]:
stat_ind_vs_aus = pd.read_csv('stats_ind_vs_aus.csv',header=0)
cols_to_merge = ['Mat', 'NO', 'HS', 'Ave', 'SR', '0s', 'From', 'To', 'bat_prob', 'match_per_year', '100_rank', '50_rank', 'Inns_rank',
 'Inns_bowl', 'Mdns_bowl', 'Ave_bowl', 'Econ_bowl', '5_bowl', '10_bowl', 'Ct_bowl', 'St_bowl', 'BBI_wkts_bowl',
 'BBI_runs_bowl', 'To_bowl', 'bowl_prob_bowl', 'Mdns_rank_bowl',
 'BBI_wkts_rank_bowl',
 'performance_rank_bowl']
df_merged2 = pd.merge(df_merged1,
                      stat_ind_vs_aus[cols_to_merge],
                      left_on='Player1',right_on='Player1',how='left')

In [627]:
corr_matrix = df_merged1.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
# Drop features 
to_drop

#df.drop(df[to_drop], axis=1)

C:\Users\Acer\AppData\Local\Temp\ipykernel_13556\4289425895.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df_merged1.corr().abs()
C:\Users\Acer\AppData\Local\Temp\ipykernel_13556\4289425895.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


['week',
 '50',
 'Inns',
 'Mat',
 'avg_out_notout_encode',
 'avg_batted_in_match_encode',
 'avg_runs_conceded_1_encode']

In [491]:
def corrFilter(x: pd.DataFrame, bound: float):
    xCorr = x.corr()
    xFiltered = xCorr[((xCorr >= bound) | (xCorr <= -bound)) & (xCorr !=1.000)]
    xFlattened = xFiltered.unstack().sort_values().drop_duplicates()
    return xFlattened

corrFilter(df_merged1, .9)


C:\Users\Acer\AppData\Local\Temp\ipykernel_13556\521218517.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  xCorr = x.corr()


avg_batted_in_match_encode  avg_out_notout_encode        -0.983023
avg_runs_scored_1_encode    avg_out_notout_encode        -0.971142
avg_runs_conceded_1_encode  avg_runs_scored_1_encode     -0.909962
out_notout                  batted_in_match              -0.907918
Ave                         avg_wickets_1_encode         -0.903235
avg_wickets_1_encode        avg_bowled_in_match_encode    0.918794
Inns                        100                           0.927953
avg_runs_conceded_1_encode  avg_wickets_1_encode          0.934938
100                         50                            0.961979
50                          Inns                          0.968360
Mat                         Inns                          0.968550
avg_bowled_in_match_encode  avg_runs_conceded_1_encode    0.971427
avg_runs_scored_1_encode    avg_batted_in_match_encode    0.978928
month                       week                          0.996814
player_id                   player_id                         

In [492]:
# y_runs = np.log1p(df_merged['runs_scored_1'])
# y_wickets = np.log1p(df_merged['wickets_1'])

y_runs =df_merged1['runs_scored_1']
y_runs = np.where(y_runs<0, 0,y_runs)
y_wickets = df_merged1['wickets_1']

cols_to_drop =['player_id', 'player_name', 'runs_scored', 'wickets', 'runs_conceded','catches', 'stumpings','day','out_notout',
               'bowled_in_match','batted_in_match','catches', 'stumpings', 'match_date', 'opposition', 'match_id','wickets_1',
               'runs_scored_1', 'runs_conceded_1', 'catches_1','stumpings_1' ,'match_date_1','player1','Player1']
X = df_merged1.drop(cols_to_drop,axis=1)

In [498]:
y_runs.max()

264

In [499]:
X.columns

Index(['is_match_complete', 'year', 'month', 'week', 'match_bat_encoded',
       'match_bowl_encoded', 'day_of_the_week_code', '0s', '100', '50', 'Ave',
       'HS', 'Inns', 'Mat', 'NO', 'SR', 'avg_runs_scored_1_encode',
       'avg_out_notout_encode', 'avg_batted_in_match_encode',
       'avg_bowled_in_match_encode', 'avg_wickets_1_encode',
       'avg_runs_conceded_1_encode', 'avg_catches_1_encode',
       'avg_stumpings_1_encode', 'opposition_country_afghanistan',
       'opposition_country_australia', 'opposition_country_bangladesh',
       'opposition_country_canada', 'opposition_country_england',
       'opposition_country_hong kong', 'opposition_country_india',
       'opposition_country_ireland', 'opposition_country_kenya',
       'opposition_country_nepal', 'opposition_country_netherlands',
       'opposition_country_new zealand', 'opposition_country_pakistan',
       'opposition_country_scotland', 'opposition_country_south africa',
       'opposition_country_sri lanka', 'oppo

In [501]:
X.to_csv('model_data.csv', index=False)
#y_runs.to_csv('y_runs.csv', index=False)
y_wickets.to_csv('y_wickets.csv', index=False)
submission_data.to_csv('submission_data.csv', index=False)

In [205]:
import seaborn as sns
import matplotlib.pyplot as plt
corr = df_merged.corr()

# kot = corr[corr>=.9]
# plt.figure(figsize=(12,8))
# sns.heatmap(kot, cmap="Greens")

C:\Users\Acer\AppData\Local\Temp\ipykernel_13556\1386994461.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df_merged.corr()


In [207]:
from sklearn.metrics import mean_squared_error,mean_squared_log_error
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GridSearchCV,RandomizedSearchCV,cross_val_score,RepeatedKFold
from sklearn.preprocessing import StandardScaler,MinMaxScaler,Normalizer,RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures

import xgboost as xgb
import lightgbm as lgb
import sklearn.ensemble as ensemble
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier,RandomForestRegressor,BaggingRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,LogisticRegression,Lasso, Ridge,LogisticRegressionCV,RidgeCV,LassoCV,ElasticNetCV,OrthogonalMatchingPursuit,ElasticNet,LassoLarsCV,BayesianRidge
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC,SVR
from scipy import stats
from scipy.stats import norm, skew
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.kernel_ridge import KernelRidge

In [604]:
y = np.log1p(y_runs)
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2,random_state=23)

In [503]:
len(x_train.columns)

43

In [608]:
rf_base = RandomForestRegressor()
rf_base.fit(x_train,y_train)


rf_tuned = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=100,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, 
                      min_samples_leaf=1, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=True, random_state=None,
                      verbose=0, warm_start=False)
rf_tuned.fit(x_train,y_train)

d:\Study\Great Lakes\INtro to python\Anaconda\envs\py310\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


RandomForestRegressor(criterion='mse', max_depth=100, max_features='sqrt',
                      min_samples_split=10, n_estimators=1000, oob_score=True)

In [609]:
model_lgb_base=lgb.LGBMRegressor(objective='regression')
model_lgb_base.fit(x_train,y_train)

model_lgb_tuned=lgb.LGBMRegressor(bagging_fraction=0.8, bagging_frequency=4, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.5,
              importance_type='split', learning_rate=0.1, max_depth=50,
              min_child_samples=20, min_child_weight=30, min_data_in_leaf=70,
              min_split_gain=0.0001, n_estimators=500, n_jobs=-1,
              num_leaves=1200, objective=None, random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

model_lgb_tuned.fit(x_train,y_train)

C:\Users\Acer\AppData\Roaming\Python\Python310\site-packages\lightgbm\sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMRegressor(bagging_fraction=0.8, bagging_frequency=4, feature_fraction=0.5,
              max_depth=50, min_child_weight=30, min_data_in_leaf=70,
              min_split_gain=0.0001, n_estimators=500, num_leaves=1200,
              silent=True)

In [603]:
from sklearn.linear_model import BayesianRidge, HuberRegressor, Ridge, OrthogonalMatchingPursuit
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
import lightgbm
from lightgbm import LGBMRegressor 
from catboost import CatBoostRegressor

In [611]:
model_cb_base=CatBoostRegressor(loss_function='RMSE',n_estimators=300, verbose = 0)
model_cb_base.fit(x_train,y_train)

In [613]:
from sklearn.metrics import mean_squared_error
prediction_rfb_valid=rf_base.predict(x_valid)
prediction_rft_valid=rf_tuned.predict(x_valid)
prediction_lgbmb_valid=model_lgb_base.predict(x_valid)
prediction_lgbmt_valid=model_lgb_tuned.predict(x_valid)
prediction_cbb_valid=model_cb_base.predict(x_valid)
mse_rfb_valid = mean_squared_error(y_true=y_valid, y_pred=prediction_rfb_valid, squared=True)
mse_rft_valid = mean_squared_error(y_true=y_valid, y_pred=prediction_rft_valid, squared=True)
mse_lgbmb_valid = mean_squared_error(y_true=y_valid, y_pred=prediction_lgbmb_valid, squared=True)
mse_lgbmt_valid = mean_squared_error(y_true=y_valid, y_pred=prediction_lgbmt_valid, squared=True)
mse_cbb_valid = mean_squared_error(y_true=y_valid, y_pred=prediction_cbb_valid, squared=True)
print('MSE rfb_valid: {:.6f}'.format(mse_rfb_valid))
print('MSE rft_valid: {:.6f}'.format(mse_rft_valid))
print('MSE lgbmb_valid: {:.6f}'.format(mse_lgbmb_valid))
print('MSE lgbmt_valid: {:.6f}'.format(mse_lgbmt_valid))
print('MSE cbb_valid: {:.6f}'.format(mse_cbb_valid))

MSE rfb_valid: 2.014612
MSE rft_valid: 1.885243
MSE lgbmb_valid: 1.884586
MSE lgbmt_valid: 2.018264
MSE cbb_valid: 1.908902


In [555]:
prediction_rfb_valid1=np.where(prediction_rfb_valid<0, 0, prediction_rfb_valid)
prediction_rft_valid1=np.where(prediction_rft_valid<0, 0, prediction_rft_valid)
prediction_lgbmb_valid1=np.where(prediction_lgbmb_valid<0, 0, prediction_lgbmb_valid)
prediction_lgbmt_valid1=np.where(prediction_lgbmt_valid<0, 0, prediction_lgbmt_valid)
y_valid1 = np.where(y_valid<0,0,y_valid)

mse_rfb_valid = mean_squared_error(y_true=y_valid1, y_pred=prediction_rfb_valid1, squared=True)
mse_rft_valid = mean_squared_error(y_true=y_valid1, y_pred=prediction_rft_valid1, squared=True)
mse_lgbmb_valid = mean_squared_error(y_true=y_valid1, y_pred=prediction_lgbmb_valid1, squared=True)
mse_lgbmt_valid = mean_squared_error(y_true=y_valid1, y_pred=prediction_lgbmt_valid1, squared=True)
print('MSE rfb_valid: {:.6f}'.format(mse_rfb_valid))
print('MSE rft_valid: {:.6f}'.format(mse_rft_valid))
print('MSE lgbmb_valid: {:.6f}'.format(mse_lgbmb_valid))
print('MSE lgbmt_valid: {:.6f}'.format(mse_lgbmt_valid))

MSE rfb_valid: 2.002715
MSE rft_valid: 1.881728
MSE lgbmb_valid: 1.882684
MSE lgbmt_valid: 2.015285


In [558]:
model = RandomForestRegressor(criterion='mse', max_depth=100, max_features='sqrt',
                      min_samples_split=10, n_estimators=1000, oob_score=True)
model.fit(X,y)

d:\Study\Great Lakes\INtro to python\Anaconda\envs\py310\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


RandomForestRegressor(criterion='mse', max_depth=100, max_features='sqrt',
                      min_samples_split=10, n_estimators=1000, oob_score=True)

In [559]:
sub.columns.difference(X.columns)

Index(['player_id', 'runs', 'wickets'], dtype='object')

In [614]:
sub = submission_data.drop(['Player1', 'player_name','date', 'player_id', 'runs', 'wickets'], axis=1)
prediction_rft_valid=model_cb_base.predict(sub)
prediction_rft_run1=np.where(prediction_rft_valid<0, 0, prediction_rft_valid)

In [616]:
pd.DataFrame({'run':y_runs}).describe()

,run
count,2575.000000
mean,24.214369
std,34.103023
min,0.000000
25%,0.000000
50%,9.000000
75%,35.000000
max,264.000000


In [530]:
submission_data.head()

,player_id,runs,wickets,Player1,player_name,0s,100,50,Ave,HS,Inns,Mat,NO,SR,avg_runs_scored_1_encode,avg_out_notout_encode,avg_batted_in_match_encode,avg_bowled_in_match_encode,avg_wickets_1_encode,avg_runs_conceded_1_encode,avg_catches_1_encode,avg_stumpings_1_encode,match_bat_encoded,match_bowl_encoded,opposition_country_afghanistan,opposition_country_australia,opposition_country_bangladesh,opposition_country_canada,opposition_country_england,opposition_country_hong kong,opposition_country_india,opposition_country_ireland,opposition_country_kenya,opposition_country_nepal,opposition_country_netherlands,opposition_country_new zealand,opposition_country_pakistan,opposition_country_scotland,opposition_country_south africa,opposition_country_sri lanka,opposition_country_u.a.e.,opposition_country_west indies,opposition_country_zimbabwe,year,month,date,week,day_of_the_week_code,is_match_complete
0,1,30,2,P Cummins,Pat Cummins,8,0,0,11.37,36,49,77,17,77.61,-27.250000,1.000000,0.625000,1.0,1.875,41.500000,0.500000,0.000000,1.750000,93.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2023,10,2023-10-08,40,7,1
1,2,24,0,S Smith,Steve Smith,7,12,30,44.33,164,129,145,15,87.66,33.941176,0.294118,0.941176,0.0,0.000,0.000000,0.705882,0.000000,8.062500,0.380952,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2023,10,2023-10-08,40,7,1
2,3,45,1,A Carey,Alex Carey,3,1,8,33.59,106,65,71,11,88.48,21.517241,0.206897,0.965517,0.0,0.000,0.000000,1.137931,0.103448,10.833333,0.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2023,10,2023-10-08,40,7,1
3,4,5,2,J Inglis,Josh Inglis,1,0,1,17.87,50,8,8,0,93.46,17.875000,0.000000,1.000000,0.0,0.000,0.000000,0.250000,0.000000,74.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2023,10,2023-10-08,40,7,1
4,5,9,0,S Abbott,Sean Abbott,1,0,1,17.53,54,14,17,1,107.54,-5.066667,0.466667,0.800000,1.0,1.200,41.866667,0.466667,0.000000,4.666667,93.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2023,10,2023-10-08,40,7,1


## Wickets prediction

In [590]:
y = np.log1p(y_wickets)
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2,random_state=23)

In [591]:
rf_base = RandomForestRegressor()
rf_base.fit(x_train,y_train)


rf_tuned = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, 
                      min_samples_leaf=1, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=True, random_state=None,
                      verbose=0, warm_start=False)
rf_tuned.fit(x_train,y_train)

d:\Study\Great Lakes\INtro to python\Anaconda\envs\py310\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


RandomForestRegressor(criterion='mse', max_depth=50, max_features='sqrt',
                      min_samples_split=10, n_estimators=1000, oob_score=True)

In [592]:
model_lgb_base=lgb.LGBMRegressor(objective='regression')
model_lgb_base.fit(x_train,y_train)

model_lgb_tuned=lgb.LGBMRegressor(bagging_fraction=0.8, bagging_frequency=4, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.5,
              importance_type='split', learning_rate=0.1, max_depth=50,
              min_child_samples=20, min_child_weight=30, min_data_in_leaf=70,
              min_split_gain=0.0001, n_estimators=500, n_jobs=-1,
              num_leaves=1200, objective=None, random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

model_lgb_tuned.fit(x_train,y_train)

C:\Users\Acer\AppData\Roaming\Python\Python310\site-packages\lightgbm\sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMRegressor(bagging_fraction=0.8, bagging_frequency=4, feature_fraction=0.5,
              max_depth=50, min_child_weight=30, min_data_in_leaf=70,
              min_split_gain=0.0001, n_estimators=500, num_leaves=1200,
              silent=True)

In [593]:
prediction_rfb_valid=rf_base.predict(x_valid)
prediction_rft_valid=rf_tuned.predict(x_valid)
prediction_lgbmb_valid=model_lgb_base.predict(x_valid)
prediction_lgbmt_valid=model_lgb_tuned.predict(x_valid)
mse_rfb_valid = mean_squared_error(y_true=y_valid, y_pred=prediction_rfb_valid, squared=True)
mse_rft_valid = mean_squared_error(y_true=y_valid, y_pred=prediction_rft_valid, squared=True)
mse_lgbmb_valid = mean_squared_error(y_true=y_valid, y_pred=prediction_lgbmb_valid, squared=True)
mse_lgbmt_valid = mean_squared_error(y_true=y_valid, y_pred=prediction_lgbmt_valid, squared=True)
print('MSE rfb_valid: {:.6f}'.format(mse_rfb_valid))
print('MSE rft_valid: {:.6f}'.format(mse_rft_valid))
print('MSE lgbmb_valid: {:.6f}'.format(mse_lgbmb_valid))
print('MSE lgbmt_valid: {:.6f}'.format(mse_lgbmt_valid))

MSE rfb_valid: 0.174342
MSE rft_valid: 0.151029
MSE lgbmb_valid: 0.170166
MSE lgbmt_valid: 0.186157


In [594]:
prediction_rfb_valid1=np.where(prediction_rfb_valid<0, 0, prediction_rfb_valid)
prediction_rft_valid1=np.where(prediction_rft_valid<0, 0, prediction_rft_valid)
prediction_lgbmb_valid1=np.where(prediction_lgbmb_valid<0, 0, prediction_lgbmb_valid)
prediction_lgbmt_valid1=np.where(prediction_lgbmt_valid<0, 0, prediction_lgbmt_valid)
y_valid1 = np.where(y_valid<0,0,y_valid)

mse_rfb_valid = mean_squared_error(y_true=y_valid1, y_pred=prediction_rfb_valid1, squared=True)
mse_rft_valid = mean_squared_error(y_true=y_valid1, y_pred=prediction_rft_valid1, squared=True)
mse_lgbmb_valid = mean_squared_error(y_true=y_valid1, y_pred=prediction_lgbmb_valid1, squared=True)
mse_lgbmt_valid = mean_squared_error(y_true=y_valid1, y_pred=prediction_lgbmt_valid1, squared=True)
print('MSE rfb_valid: {:.6f}'.format(mse_rfb_valid))
print('MSE rft_valid: {:.6f}'.format(mse_rft_valid))
print('MSE lgbmb_valid: {:.6f}'.format(mse_lgbmb_valid))
print('MSE lgbmt_valid: {:.6f}'.format(mse_lgbmt_valid))

MSE rfb_valid: 0.174342
MSE rft_valid: 0.151029
MSE lgbmb_valid: 0.169898
MSE lgbmt_valid: 0.185148


In [595]:
model = RandomForestRegressor(criterion='mse', max_depth=50, max_features='sqrt',
                      min_samples_split=10, n_estimators=1000, oob_score=True)
model.fit(X,y)

d:\Study\Great Lakes\INtro to python\Anaconda\envs\py310\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


RandomForestRegressor(criterion='mse', max_depth=50, max_features='sqrt',
                      min_samples_split=10, n_estimators=1000, oob_score=True)

In [596]:
#sub.columns.difference(X.columns)
sub = submission_data.drop(['Player1', 'player_name','date', 'player_id', 'runs', 'wickets'], axis=1)
prediction_rft_valid=model.predict(sub)
prediction_rft_wkts1=np.where(prediction_rft_valid<0, 0, prediction_rft_valid)

d:\Study\Great Lakes\INtro to python\Anaconda\envs\py310\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [601]:
(np.expm1(prediction_rft_wkts1))

array([0.49673816, 0.67645048, 0.66793445, 0.50606462, 0.46730475,
       0.64539928, 0.39175226, 0.49167191, 0.51138316, 0.53099539,
       0.53554083, 0.53357399, 0.66997661, 0.49397237, 0.49441   ,
       0.8095376 , 0.69823078, 0.78870399, 0.80097664, 0.60993871,
       0.81978541, 0.66735829, 0.60149577, 0.58968789, 0.61581035,
       0.59018546, 0.60522662, 0.6052211 , 0.56153235, 0.72159432])

In [602]:
sub = pd.DataFrame()
sub['player_id'] = submission_data['player_id']
sub['runs']= np.round(np.expm1(prediction_rft_run1[:,]))
sub['wickets']= np.round(np.expm1(prediction_rft_wkts1[:,]))
sub.to_csv('submission2.csv', index=False)

In [625]:
submission_data.columns


Index(['player_id', 'runs', 'wickets', 'Player1', 'player_name', '0s', '100',
       '50', 'Ave', 'HS', 'Inns', 'Mat', 'NO', 'SR',
       'avg_runs_scored_1_encode', 'avg_out_notout_encode',
       'avg_batted_in_match_encode', 'avg_bowled_in_match_encode',
       'avg_wickets_1_encode', 'avg_runs_conceded_1_encode',
       'avg_catches_1_encode', 'avg_stumpings_1_encode', 'match_bat_encoded',
       'match_bowl_encoded', 'opposition_country_afghanistan',
       'opposition_country_australia', 'opposition_country_bangladesh',
       'opposition_country_canada', 'opposition_country_england',
       'opposition_country_hong kong', 'opposition_country_india',
       'opposition_country_ireland', 'opposition_country_kenya',
       'opposition_country_nepal', 'opposition_country_netherlands',
       'opposition_country_new zealand', 'opposition_country_pakistan',
       'opposition_country_scotland', 'opposition_country_south africa',
       'opposition_country_sri lanka', 'opposition_count

In [626]:
submission_data[['Player1', 'avg_batted_in_match_encode', 'avg_bowled_in_match_encode','Ave']]

,Player1,avg_batted_in_match_encode,avg_bowled_in_match_encode,Ave
0,P Cummins,0.625000,1.000000,11.37
1,S Smith,0.941176,0.000000,44.33
2,A Carey,0.965517,0.000000,33.59
3,J Inglis,1.000000,0.000000,17.87
4,S Abbott,0.800000,1.000000,17.53
5,M Labuschagne,0.960000,0.280000,37.29
6,C Green,0.842105,1.000000,37.90
7,J Hazlewood,0.400000,1.000000,23.40
8,T Head,1.000000,0.375000,41.28
9,M Marsh,1.000000,0.368421,34.32
